In [1]:
import pandas as pd

In [21]:
df = pd.read_hdf(r'C:\Users\Linds\Repos\East_River\data\processed\calander_weather_scada.h5')



In [22]:


df.isnull().sum()


df.isnull().sum().sum()



missing_counts = df.isnull().sum()
sorted_missing = missing_counts.sort_values(ascending=False)
print("Missing values per column:", sorted_missing)


Missing values per column: OnLine_Load_MW                      9396
Unnamed: 0                          7656
Timestamp                           7656
date                                   0
diffuse_radiation                      0
direct_radiation                       0
pressure_msl                           0
surface_pressure                       0
blended_precip                         0
latitude                               0
longitude                              0
noaa_station_name                      0
location                               0
relative_humidity_2m                   0
Load_Control_KW                        0
Load_Control_MW                        0
Estimated_Online_Load_Control_MW       0
Control_Threshold_MW                   0
day_of_week                            0
local_time                             0
is_weekend                             0
global_tilted_irradiance               0
apparent_temperature                   0
datetime                      

In [3]:
df.head(300)

,location,datetime,temperature,wind_speed,wind_degree,precip,humidity,visibility,pressure,cloudcover,...,Timestamp,OnLine_Load_MW,Load_Control_KW,Load_Control_MW,Estimated_Online_Load_Control_MW,Control_Threshold_MW,day_of_week,local_time,is_weekend,is_holiday
0,"Aberdeen, South Dakota",2020-12-31 06:00:00+00:00,12.0,7.0,197.0,0.0,90.0,6.0,30.120,69.0,...,NaT,NaN,5077.996078,5.077996,500.291922,593.103098,Thursday,2020-12-31 00:00:00-06:00,False,False
1,"Aberdeen, South Dakota",2020-12-31 06:30:00+00:00,11.0,7.0,196.0,0.0,90.0,6.0,30.105,61.0,...,NaT,NaN,5077.996078,5.077996,500.291922,593.103098,Thursday,2020-12-31 00:30:00-06:00,False,False
2,"Aberdeen, South Dakota",2020-12-31 07:00:00+00:00,10.0,7.0,195.0,0.0,90.0,6.0,30.090,53.0,...,NaT,NaN,5077.996078,5.077996,500.291922,593.103098,Thursday,2020-12-31 01:00:00-06:00,False,False
3,"Aberdeen, South Dakota",2020-12-31 07:30:00+00:00,10.0,7.0,194.0,0.0,89.5,6.0,30.090,45.5,...,NaT,NaN,5077.996078,5.077996,500.291922,593.103098,Thursday,2020-12-31 01:30:00-06:00,False,False
4,"Aberdeen, South Dakota",2020-12-31 08:00:00+00:00,10.0,7.0,193.0,0.0,89.0,6.0,30.090,38.0,...,NaT,NaN,5077.996078,5.077996,500.291922,593.103098,Thursday,2020-12-31 02:00:00-06:00,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,"Aberdeen, South Dakota",2021-01-06 09:30:00+00:00,28.0,6.0,45.0,0.0,98.0,1.0,30.045,100.0,...,2021-01-06 09:30:00+00:00,525.68,728.000000,0.728000,526.408000,575.000000,Wednesday,2021-01-06 03:30:00-06:00,False,False
296,"Aberdeen, South Dakota",2021-01-06 10:00:00+00:00,28.0,6.0,40.0,0.0,98.0,1.0,30.060,100.0,...,2021-01-06 10:00:00+00:00,520.74,728.000000,0.728000,521.468000,575.000000,Wednesday,2021-01-06 04:00:00-06:00,False,False
297,"Aberdeen, South Dakota",2021-01-06 10:30:00+00:00,28.0,6.0,35.0,0.0,98.0,1.0,30.060,100.0,...,2021-01-06 10:30:00+00:00,510.70,728.000000,0.728000,511.428000,575.000000,Wednesday,2021-01-06 04:30:00-06:00,False,False
298,"Aberdeen, South Dakota",2021-01-06 11:00:00+00:00,28.0,6.0,30.0,0.0,98.0,1.0,30.060,100.0,...,2021-01-06 11:00:00+00:00,505.73,728.000000,0.728000,506.458000,575.000000,Wednesday,2021-01-06 05:00:00-06:00,False,False


#Ideas for feature engineering:

#- Time Features
    - Extract hour, day, month, year, and minute from the datetime or local_time columns.
    - Use day_of_week, is_weekend, and is_holiday to capture recurring patterns.
    - Derive features like “time since start of day” or “sin/cos” transformations to capture cyclical behavior.

#- Lag and Window Features
    - Create lag features for Estimated_Online_Load_Control_MW (e.g., previous 1, 3, or 6 hours).
    - Compute rolling statistics (mean, standard deviation) for short-term and mid-term intervals to capture trends.

#- Weather Interactions
    - Include temperature, humidity, precipitation, wind_speed, cloudcover, etc. as features.
    - Generate cross features between weather variables (e.g., interactions between temperature and humidity).

#- Categorical Encoding
    - Encode categorical variables such as location, weather_descriptions, wind_dir, noaa_station_name, and day_of_week.
    - Consider one-hot encoding or target encoding if there are too many categories.

#- Other Considerations
    - Remove or combine highly correlated features.
    - Feature scaling might be helpful for some XGBoost setups.
    - Experiment with interaction features or polynomial features if needed.  

Capture seasonality, trends, and the impact of weather conditions on load, which should improve the performance of XGBoost model for forecasting the Estimated Online Load (MW) for the upcoming 24–72 hours.

## Time Features

In [6]:

# Create time features from the 'local_time' column
df['hour'] = df['local_time'].dt.hour
df['minute'] = df['local_time'].dt.minute
df['day'] = df['local_time'].dt.day
df['month'] = df['local_time'].dt.month
df['year'] = df['local_time'].dt.year

# Calculate minutes since midnight
df['minutes_since_midnight'] = df['hour'] * 60 + df['minute']

df.head()

,location,datetime,temperature,wind_speed,wind_degree,precip,humidity,visibility,pressure,cloudcover,...,day_of_week,local_time,is_weekend,is_holiday,hour,minute,day,month,year,minutes_since_midnight
0,"Aberdeen, South Dakota",2020-12-31 06:00:00+00:00,12.0,7.0,197.0,0.0,90.0,6.0,30.120,69.0,...,Thursday,2020-12-31 00:00:00-06:00,False,False,0,0,31,12,2020,0
1,"Aberdeen, South Dakota",2020-12-31 06:30:00+00:00,11.0,7.0,196.0,0.0,90.0,6.0,30.105,61.0,...,Thursday,2020-12-31 00:30:00-06:00,False,False,0,30,31,12,2020,30
2,"Aberdeen, South Dakota",2020-12-31 07:00:00+00:00,10.0,7.0,195.0,0.0,90.0,6.0,30.090,53.0,...,Thursday,2020-12-31 01:00:00-06:00,False,False,1,0,31,12,2020,60
3,"Aberdeen, South Dakota",2020-12-31 07:30:00+00:00,10.0,7.0,194.0,0.0,89.5,6.0,30.090,45.5,...,Thursday,2020-12-31 01:30:00-06:00,False,False,1,30,31,12,2020,90
4,"Aberdeen, South Dakota",2020-12-31 08:00:00+00:00,10.0,7.0,193.0,0.0,89.0,6.0,30.090,38.0,...,Thursday,2020-12-31 02:00:00-06:00,False,False,2,0,31,12,2020,120


In [7]:
import numpy as np

# Encode the hour feature using sine and cosine transforms to capture cyclical patterns.
df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)

df.head()

,location,datetime,temperature,wind_speed,wind_degree,precip,humidity,visibility,pressure,cloudcover,...,is_weekend,is_holiday,hour,minute,day,month,year,minutes_since_midnight,hour_sin,hour_cos
0,"Aberdeen, South Dakota",2020-12-31 06:00:00+00:00,12.0,7.0,197.0,0.0,90.0,6.0,30.120,69.0,...,False,False,0,0,31,12,2020,0,0.000000,1.000000
1,"Aberdeen, South Dakota",2020-12-31 06:30:00+00:00,11.0,7.0,196.0,0.0,90.0,6.0,30.105,61.0,...,False,False,0,30,31,12,2020,30,0.000000,1.000000
2,"Aberdeen, South Dakota",2020-12-31 07:00:00+00:00,10.0,7.0,195.0,0.0,90.0,6.0,30.090,53.0,...,False,False,1,0,31,12,2020,60,0.258819,0.965926
3,"Aberdeen, South Dakota",2020-12-31 07:30:00+00:00,10.0,7.0,194.0,0.0,89.5,6.0,30.090,45.5,...,False,False,1,30,31,12,2020,90,0.258819,0.965926
4,"Aberdeen, South Dakota",2020-12-31 08:00:00+00:00,10.0,7.0,193.0,0.0,89.0,6.0,30.090,38.0,...,False,False,2,0,31,12,2020,120,0.500000,0.866025


In [8]:
# Create a numeric day-of-week column from 'local_time' (Monday=0, ..., Sunday=6)
df['day_of_week_num'] = df['local_time'].dt.dayofweek

# Create a binary weekend flag: 1 for Saturday (5) or Sunday (6), else 0
df['weekend_flag'] = (df['day_of_week_num'] >= 5).astype(int)

# Display the new columns for confirmation
df[['day_of_week_num', 'weekend_flag']].head()

,day_of_week_num,weekend_flag
0,3,0
1,3,0
2,3,0
3,3,0
4,3,0


In [9]:
df.head()

,location,datetime,temperature,wind_speed,wind_degree,precip,humidity,visibility,pressure,cloudcover,...,hour,minute,day,month,year,minutes_since_midnight,hour_sin,hour_cos,day_of_week_num,weekend_flag
0,"Aberdeen, South Dakota",2020-12-31 06:00:00+00:00,12.0,7.0,197.0,0.0,90.0,6.0,30.120,69.0,...,0,0,31,12,2020,0,0.000000,1.000000,3,0
1,"Aberdeen, South Dakota",2020-12-31 06:30:00+00:00,11.0,7.0,196.0,0.0,90.0,6.0,30.105,61.0,...,0,30,31,12,2020,30,0.000000,1.000000,3,0
2,"Aberdeen, South Dakota",2020-12-31 07:00:00+00:00,10.0,7.0,195.0,0.0,90.0,6.0,30.090,53.0,...,1,0,31,12,2020,60,0.258819,0.965926,3,0
3,"Aberdeen, South Dakota",2020-12-31 07:30:00+00:00,10.0,7.0,194.0,0.0,89.5,6.0,30.090,45.5,...,1,30,31,12,2020,90,0.258819,0.965926,3,0
4,"Aberdeen, South Dakota",2020-12-31 08:00:00+00:00,10.0,7.0,193.0,0.0,89.0,6.0,30.090,38.0,...,2,0,31,12,2020,120,0.500000,0.866025,3,0


In [10]:
print(df[['is_weekend', 'weekend_flag']].head())

   is_weekend  weekend_flag
0       False             0
1       False             0
2       False             0
3       False             0
4       False             0


In [11]:
# Remove the weekend_flag column while keeping the is_weekend column
df.drop(columns=['weekend_flag'], inplace=True)

# Verify the change by displaying the first few rows
df.head()

,location,datetime,temperature,wind_speed,wind_degree,precip,humidity,visibility,pressure,cloudcover,...,is_holiday,hour,minute,day,month,year,minutes_since_midnight,hour_sin,hour_cos,day_of_week_num
0,"Aberdeen, South Dakota",2020-12-31 06:00:00+00:00,12.0,7.0,197.0,0.0,90.0,6.0,30.120,69.0,...,False,0,0,31,12,2020,0,0.000000,1.000000,3
1,"Aberdeen, South Dakota",2020-12-31 06:30:00+00:00,11.0,7.0,196.0,0.0,90.0,6.0,30.105,61.0,...,False,0,30,31,12,2020,30,0.000000,1.000000,3
2,"Aberdeen, South Dakota",2020-12-31 07:00:00+00:00,10.0,7.0,195.0,0.0,90.0,6.0,30.090,53.0,...,False,1,0,31,12,2020,60,0.258819,0.965926,3
3,"Aberdeen, South Dakota",2020-12-31 07:30:00+00:00,10.0,7.0,194.0,0.0,89.5,6.0,30.090,45.5,...,False,1,30,31,12,2020,90,0.258819,0.965926,3
4,"Aberdeen, South Dakota",2020-12-31 08:00:00+00:00,10.0,7.0,193.0,0.0,89.0,6.0,30.090,38.0,...,False,2,0,31,12,2020,120,0.500000,0.866025,3


In [12]:
print(df.columns.tolist())

['location', 'datetime', 'temperature', 'wind_speed', 'wind_degree', 'precip', 'humidity', 'visibility', 'pressure', 'cloudcover', 'heatindex', 'windchill', 'windgust', 'feelslike', 'uv_index', 'obs_tms_lcl', 'wind_dir', 'weather_descriptions', 'temperature_2m', 'precipitation', 'snowfall', 'snow_depth', 'apparent_temperature', 'relative_humidity_2m', 'global_tilted_irradiance', 'diffuse_radiation', 'direct_radiation', 'pressure_msl', 'surface_pressure', 'blended_precip', 'latitude', 'longitude', 'noaa_station_name', 'date', 'Unnamed: 0', 'Timestamp', 'OnLine_Load_MW', 'Load_Control_KW', 'Load_Control_MW', 'Estimated_Online_Load_Control_MW', 'Control_Threshold_MW', 'day_of_week', 'local_time', 'is_weekend', 'is_holiday', 'hour', 'minute', 'day', 'month', 'year', 'minutes_since_midnight', 'hour_sin', 'hour_cos', 'day_of_week_num']


## Lag & Rolling Stats

Time

In [13]:
# Ensure the dataframe is sorted by the local_time column
df.sort_values('local_time', inplace=True)

# Create lag features for the load control value.
# Adjust the shift values as needed for your time resolution.
df['load_lag_1'] = df['Estimated_Online_Load_Control_MW'].shift(1)
df['load_lag_2'] = df['Estimated_Online_Load_Control_MW'].shift(2)
df['load_lag_3'] = df['Estimated_Online_Load_Control_MW'].shift(3)

# Optionally, view the new columns to verify
df[['Estimated_Online_Load_Control_MW', 'load_lag_1', 'load_lag_2', 'load_lag_3']].head()

,Estimated_Online_Load_Control_MW,load_lag_1,load_lag_2,load_lag_3
0,500.291922,NaN,NaN,NaN
3426815,500.291922,500.291922,NaN,NaN
3356880,500.291922,500.291922,500.291922,NaN
9581095,500.291922,500.291922,500.291922,500.291922
3286945,500.291922,500.291922,500.291922,500.291922


In [14]:
# Count missing values per column
nan_counts = df.isnull().sum()
print(nan_counts)

# Total number of NaNs in the entire DataFrame
total_nans = df.isnull().sum().sum()
print("Total number of NaN values:", total_nans)

location                               0
datetime                               0
temperature                            0
wind_speed                             0
wind_degree                            0
precip                                 0
humidity                               0
visibility                             0
pressure                               0
cloudcover                             0
heatindex                              0
windchill                              0
windgust                               0
feelslike                              0
uv_index                               0
obs_tms_lcl                            0
wind_dir                               0
weather_descriptions                   0
temperature_2m                         0
precipitation                          0
snowfall                               0
snow_depth                             0
apparent_temperature                   0
relative_humidity_2m                   0
global_tilted_ir

In [15]:

# Impute missing values in lag features with the actual load value at the same row.
impute_cols = ['load_lag_1', 'load_lag_2', 'load_lag_3']
for col in impute_cols:
    df[col].fillna(df['Estimated_Online_Load_Control_MW'], inplace=True)

# Verify that missing values have been imputed.
print(df[impute_cols].isnull().sum())


load_lag_1    0
load_lag_2    0
load_lag_3    0
dtype: int64


Rolling Averages

In [16]:
# Create rolling average features for the column stored in 'col'
# For example, here we generate rolling mean and standard deviation for different time windows

# Rolling mean and std for a 3-period window
df[f'{col}_rolling_mean_3'] = df[col].rolling(window=3, min_periods=1).mean()
df[f'{col}_rolling_std_3'] = df[col].rolling(window=3, min_periods=1).std()

# Rolling mean and std for a 6-period window
df[f'{col}_rolling_mean_6'] = df[col].rolling(window=6, min_periods=1).mean()
df[f'{col}_rolling_std_6'] = df[col].rolling(window=6, min_periods=1).std()

# Rolling mean and std for a 12-period window
df[f'{col}_rolling_mean_12'] = df[col].rolling(window=12, min_periods=1).mean()
df[f'{col}_rolling_std_12'] = df[col].rolling(window=12, min_periods=1).std()

# Display the newly created features to verify
df[[f'{col}_rolling_mean_3', f'{col}_rolling_std_3',
    f'{col}_rolling_mean_6', f'{col}_rolling_std_6',
    f'{col}_rolling_mean_12', f'{col}_rolling_std_12']].head()

,load_lag_3_rolling_mean_3,load_lag_3_rolling_std_3,load_lag_3_rolling_mean_6,load_lag_3_rolling_std_6,load_lag_3_rolling_mean_12,load_lag_3_rolling_std_12
0,500.291922,NaN,500.291922,NaN,500.291922,NaN
3426815,500.291922,0.0,500.291922,0.0,500.291922,0.0
3356880,500.291922,0.0,500.291922,0.0,500.291922,0.0
9581095,500.291922,0.0,500.291922,0.0,500.291922,0.0
3286945,500.291922,0.0,500.291922,0.0,500.291922,0.0


In [17]:
df.head()

,location,datetime,temperature,wind_speed,wind_degree,precip,humidity,visibility,pressure,cloudcover,...,day_of_week_num,load_lag_1,load_lag_2,load_lag_3,load_lag_3_rolling_mean_3,load_lag_3_rolling_std_3,load_lag_3_rolling_mean_6,load_lag_3_rolling_std_6,load_lag_3_rolling_mean_12,load_lag_3_rolling_std_12
0,"Aberdeen, South Dakota",2020-12-31 06:00:00+00:00,12.0,7.0,197.0,0.0,90.0,6.0,30.12,69.0,...,3,500.291922,500.291922,500.291922,500.291922,NaN,500.291922,NaN,500.291922,NaN
3426815,"Eagle, South Dakota",2020-12-31 06:00:00+00:00,10.0,6.0,157.0,0.0,86.0,6.0,30.09,25.0,...,3,500.291922,500.291922,500.291922,500.291922,0.0,500.291922,0.0,500.291922,0.0
3356880,"Dolton, South Dakota",2020-12-31 06:00:00+00:00,9.0,4.0,199.0,0.0,86.0,6.0,30.21,74.0,...,3,500.291922,500.291922,500.291922,500.291922,0.0,500.291922,0.0,500.291922,0.0
9581095,"Rousseau, South Dakota",2020-12-31 06:00:00+00:00,14.0,8.0,180.0,0.0,84.0,6.0,30.09,53.0,...,3,500.291922,500.291922,500.291922,500.291922,0.0,500.291922,0.0,500.291922,0.0
3286945,"Doland, South Dakota",2020-12-31 06:00:00+00:00,9.0,7.0,156.0,0.0,85.0,6.0,30.12,74.0,...,3,500.291922,500.291922,500.291922,500.291922,0.0,500.291922,0.0,500.291922,0.0


In [18]:
print("Total number of NaN values:", total_nans)

Total number of NaN values: 24714


In [19]:
# Check for missing values in each column
missing_counts = df.isnull().sum()
print(missing_counts)

# Calculate and print the total number of missing values in the DataFrame
total_missing = missing_counts.sum()
print("Total number of missing values:", total_missing)

location                      0
datetime                      0
temperature                   0
wind_speed                    0
wind_degree                   0
                             ..
load_lag_3_rolling_std_3      1
load_lag_3_rolling_mean_6     0
load_lag_3_rolling_std_6      1
load_lag_3_rolling_mean_12    0
load_lag_3_rolling_std_12     1
Length: 63, dtype: int64
Total number of missing values: 24711


In [20]:
# Display the missing counts for each column, sorted in descending order.
sorted_missing = missing_counts.sort_values(ascending=False)
print("Missing values per column:")
print(sorted_missing)

Missing values per column:
OnLine_Load_MW               9396
Unnamed: 0                   7656
Timestamp                    7656
load_lag_3_rolling_std_12       1
load_lag_3_rolling_std_6        1
                             ... 
direct_radiation                0
pressure_msl                    0
surface_pressure                0
blended_precip                  0
longitude                       0
Length: 63, dtype: int64
